In [ ]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import MetaData
from sqlalchemy import Table, select, func, and_, insert, delete, update, or_
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

In [ ]:
import json

sql_credentials = json.load(open('/root/thomas/sqlcredentials.json'))

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                                         sql_credentials["host"], sql_credentials["port"],
                                         sql_credentials["database"]))


In [ ]:
session_class = sessionmaker(bind=engine)
session = session_class()

Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
keypoints = Base.classes.keypoint_annotations
biomass = Base.classes.biomass_computations
detections = Base.classes.fish_detections

In [ ]:
results = session.query(keypoints, biomass, detections) \
                 .filter(biomass.keypoint_annotation_id == keypoints.id) \
                 .filter(keypoints.fish_detection_id == detections.id) \
                 .filter(detections.captured_at >= '2019-04-27') \
                 .all()

In [ ]:
print(len(results))

Calculate pairwise distances

In [ ]:
from aquabyte.optics import convert_to_world_point, depth_from_disp
import numpy as np

In [ ]:
def kp2world(keypoints, params, order):
    left_kp = keypoints['leftCrop']
    right_kp = keypoints['rightCrop']
    
    out = []
    for kpname in order:
        lkp = [kp for kp in left_kp if kp['keypointType'] == kpname][0]
        rkp = [kp for kp in right_kp if kp['keypointType'] == kpname][0]
        leftx = lkp['xFrame']
        lefty = lkp['yFrame']
        rightx = rkp['xFrame']
        
        disp = leftx - rightx
        depth = depth_from_disp(disp, params)
        world = convert_to_world_point(leftx, lefty, depth, params)
        out.append({'name': kpname, 'coord': world})
        
#     for kp in left_kp:
#         name = kp['keypointType']
#         leftx = kp['xFrame']
#         lefty = kp['yFrame']
#         for kp1 in right_kp:
#             name1 = kp1['keypointType']
#             if name1 == name:
#                 rightx = kp1['xFrame']
#                 break

    return out

In [ ]:
def calculate_pwd(worldkps):
    out = {}
    for (i, kp) in enumerate(worldkps):
        for (j, kp1) in enumerate(worldkps[i+1:]):
            dist_name = kp['name'] + '-' + kp1['name']
            dist = np.linalg.norm(kp['coord'] - kp1['coord'])
            out[dist_name] = dist
    return out

In [ ]:
order = ['UPPER_LIP',
 'EYE',
 'DORSAL_FIN',
 'ADIPOSE_FIN',
 'TAIL_NOTCH',
 'ANAL_FIN',
 'PELVIC_FIN',
 'PECTORAL_FIN']

In [ ]:
parsed_results = []
for r in results:
    camera_params = {"PIXEL_COUNT_WIDTH": r[2].camera_metadata['pixelCountWidth'],
                     "PIXEL_COUNT_HEIGHT": r[2].camera_metadata['pixelCountHeight'],
                     "IMAGE_SENSOR_HEIGHT": r[2].camera_metadata['imageSensorHeight'],
                     "IMAGE_SENSOR_WIDTH": r[2].camera_metadata['imageSensorWidth'],
                     "FOCAL_LENGTH": r[2].camera_metadata['focalLength'],
                     "FOCAL_LENGTH_PIXEL": r[2].camera_metadata['focalLengthPixel'],
                     "BASELINE": r[2].camera_metadata['baseline'],
                    }
    keypoints = r[0].keypoints
    if not keypoints:
        continue
    if 'leftCrop' not in keypoints:
        continue
    biomass = r[1].estimated_biomass_g
    
    worldkps = kp2world(keypoints, camera_params, order)
    if len(worldkps) < 8:
        continue
    pairwise_distances = calculate_pwd(worldkps)
    
    parsed_results.append([camera_params, keypoints, biomass, pairwise_distances, 
                           r[2].id, r[2].created_at, r[2].left_image_url, r[2].right_image_url, 
                           r[0].is_qa, r[2].camera_metadata, r[2].site_id, r[2].pen_id, r[1].keypoint_annotation_id])

Some plots

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
biomasses = [r[2] for r in parsed_results]
plt.hist(biomasses)
plt.title('Weight distribution (g) - from db')
plt.show()

In [ ]:
distances = {}
for r in parsed_results:
    for (dname, d) in r[3].items():
        if dname not in distances:
            distances[dname] = []
        distances[dname].append(d)

In [ ]:
for (k, v) in distances.items():
    plt.hist(v)
    plt.title(k)
    plt.show()
    
    plt.scatter(np.array(v)*100, biomasses)
    plt.xlabel('Distance in cm')
    plt.show()
    print(np.mean(v), np.std(v))
    print(np.where(np.abs(v - np.mean(v)) > 2*np.std(v)))
    print('#'*50)

In [ ]:
import cv2

In [ ]:
# test = parsed_results[123]
# lkps = test[1]['leftCrop']
# rkps = test[1]['rightCrop']

In [ ]:
# plt.figure(figsize=(20, 10))
# plt.imshow(cv2.imread('/root/data/rds/left_frame_crop_598_102_3154_2030.jpg'))
# for kp in lkps:
#     plt.scatter(kp['xCrop'], kp['yCrop'])
#     plt.text(kp['xCrop'], kp['yCrop'], kp['keypointType'], color='white')
# plt.show()

In [ ]:
# plt.figure(figsize=(20, 10))
# plt.imshow(cv2.imread('/root/data/rds/right_frame_crop_142_67_2610_1971.jpg'))
# for kp in rkps:
#     plt.scatter(kp['xCrop'], kp['yCrop'])
#     plt.text(kp['xCrop'], kp['yCrop'], kp['keypointType'], color='white')
# plt.show()

Save in the same format

In [ ]:
from datetime import datetime

In [ ]:
target_format = json.load(open("/root/data/gtsf_phase_I/2019-02-26/2019-02-26_cogito_annotations.json"))

In [ ]:
new = []
for (ts, res) in enumerate(parsed_results):
    tmpleft = {'timestamp': ts, 
               "species": 'salmon',
               'kfactor': 1.0,
               'Label': {},
               'local_path': res[6],
               'site_id': res[10],
               'pen_id': res[11],
               'keypoint_annotation_id': res[12]}
    tmpright = {'timestamp': ts, 
               "species": 'salmon',
               'kfactor': 1.0,
               'Label': {},
               'local_path': res[7],
               'site_id': res[10],
               'pen_id': res[11],
               'keypoint_annotation_id': res[12]}
    
    for lkp in res[1]['leftCrop']:
        tmpleft['Label'][lkp['keypointType']] = [{'geometry': {'x': lkp['xFrame'], 'y': lkp['yFrame']}}]
        
    for rkp in res[1]['rightCrop']:
        tmpright['Label'][rkp['keypointType']] = [{'geometry': {'x': rkp['xFrame'], 'y': rkp['yFrame']}}]
    
    new.append(tmpleft)
    new.append(tmpright)

In [ ]:
with open('/root/data/rds/formatted.json', 'w') as f:
    json.dump(new, f)

In [ ]:
res[0]

Biomass production code

In [ ]:
def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = PIXEL_COUNT_WIDTH / 2.0  
    image_center_y = PIXEL_COUNT_HEIGHT / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / 4096)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / 3000)
    print(sensor_x, sensor_z)
    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])

In [ ]:
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, exc

def euclidean_distance(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)**0.5


def convert_to_world_point(x, y, d, parameters):
    """ from pixel coordinates to world coordinates """
    # get relevant parameters
    pixel_count_height = 3000 # parameters["pixelCountWidth"]
    pixel_count_width = 4096 #parameters["pixelCountHeight"]
    sensor_width = parameters["imageSensorWidth"]
    sensor_height = parameters["imageSensorHeight"]
    focal_length = parameters["focalLength"]

    image_center_x = pixel_count_width / 2.0
    image_center_y = pixel_count_height / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (sensor_width / pixel_count_width)
    sensor_z = px_z * (sensor_height / pixel_count_height)
#     print(image_center_x, image_center_y, px_x, px_z, sensor_x, sensor_z)
    # now move to world coordinates
    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
    return np.array([world_x, world_y, world_z])


def depth_from_disp(disp, parameters):
    """ calculate the depth of the point based on the disparity value """
    focal_length_pixel = parameters["focalLengthPixel"]

    baseline = parameters["baseline"]
    depth = focal_length_pixel * baseline / np.array(disp)
    return depth


def pixel2world(left_crop, right_crop, parameters):
    """2D pixel coordinates to 3D world coordinates"""

    # first create a dic with crop keypoints
    image_coordinates = {"leftCrop": {},
                         "rightCrop": {}}
    for keypoint in left_crop:
        name = keypoint["keypointType"]
        image_coordinates["leftCrop"][name] = [keypoint["xFrame"], keypoint["yFrame"]]
    for keypoint in right_crop:
        name = keypoint["keypointType"]
        image_coordinates["rightCrop"][name] = [keypoint["xFrame"], keypoint["yFrame"]]
    
    # then loop through the right crop keypoints and calculate the world coordinates
    world_coordinates = {}
    for keypoint in left_crop:
        name = keypoint["keypointType"]
        disparity = image_coordinates["leftCrop"][name][0] - image_coordinates["rightCrop"][name][0]
        depth = depth_from_disp(disparity, parameters)
        
        world_point = convert_to_world_point(image_coordinates["leftCrop"][name][0],
                                             image_coordinates["leftCrop"][name][1],
                                             depth,
                                             parameters)
#         print(image_coordinates["leftCrop"][name], depth, world_point)
        world_coordinates[name] = world_point
    return world_coordinates


def coord2biomass(world_keypoints, blender):
    """from coordinates to biomass"""

    # mapping helps for consistency with the kp order
    reverse_mapping = blender["reverse_mapping"]
    distances = np.array(blender["distances"])
    volumes = blender["volume"]
    regression_coeff = blender["coeff"]

    # calculate pairwise distances for production coord
    # the reverse mapping insure that we listing the kp
    # in the same order
    measurements = []
    number_of_parts = len(world_keypoints)
    for k in range(number_of_parts):
        v = world_keypoints[reverse_mapping[str(k)]]
        for k0 in range(k+1, number_of_parts):
            v0 = world_keypoints[reverse_mapping[str(k0)]]
            dist = euclidean_distance(v, v0)*1000 # mm to m
            measurements.append(dist)
    print(measurements)
    # measurements = np.array(measurements)
    
    # absolute diff
    diff = np.nanmean(np.abs(distances - measurements), axis=1)
    print(diff)
    closest = np.argmin(diff)
    print(closest)
    prediction = volumes[closest]

    # here is some machine learning
    prediction = prediction*regression_coeff[0] + regression_coeff[1]
    return prediction

In [ ]:
class BiomassModel:
    def __init__(self, volumes_location, db_params):
        # open the file containing the blender volumes
        with open(volumes_location, "r") as f:
            self.blender = json.load(f)
        self.db_params = db_params
        # self.logger = logging.getLogger()

    def predict(self, jsondata):
        biomass = None
        # get the data
        try:
#             annotation_id = jsondata["annotationId"]
            parameters = jsondata["cameraParameters"]
            right_crop = jsondata["rightCrop"]
            left_crop = jsondata["leftCrop"]
#             site_id = jsondata["siteId"]
#             pen_id = jsondata["penId"]



            # pixel coordinates to world coordinates
            coordinates = pixel2world(left_crop, right_crop, parameters)

            # coordinates to biomass
            biomass = coord2biomass(coordinates, self.blender)


#             # update biomass
#             updatebiomass(biomass, annotation_id, site_id, pen_id, self.logger, user=self.db_params.user
#                         , password=self.db_params.password, host=self.db_params.host
#                           , port=self.db_params.port, database=self.db_params.db_name)
        except Exception as e:
            # self.logger.info(e)
            print("ERROR: {}".format(e))
        return coordinates, biomass

In [ ]:
bm = BiomassModel("/root/thomas/blender/volumes_all.json", None)

In [ ]:
np.array(bm.blender['distances'])[0, :]

In [ ]:
bmasses = []
for res in parsed_results:
    jsondata = {}
    jsondata['cameraParameters'] = res[-1]
    jsondata["rightCrop"] = res[1]['rightCrop']
    jsondata["leftCrop"] = res[1]['leftCrop']
    
    coord, bmass = bm.predict(jsondata)
#     print(coord)
    bmasses.append(bmass)
    break

In [ ]:
plt.hist(bmasses)
plt.show()